# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")
df.head(1)

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.21696,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.8,Four-Door Car,Medsize,NaN


1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [2]:
grouped =  df[(df["Total Claim Amount"] > 1000) & (df["Response"] == "Yes")]

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [37]:
yes = df[df["Response"] == "Yes"]
yes.groupby(["Policy", "Gender"])["Total Claim Amount"].mean().round(2)

Policy        Gender
Corporate L1  F         463.94
              M         426.48
Corporate L2  F         441.35
              M         427.99
Corporate L3  F         421.23
              M         391.61
Personal L1   F         485.13
              M         464.62
Personal L2   F         439.91
              M         459.43
Personal L3   F         447.08
              M         452.95
Special L1    F         471.34
              M         476.39
Special L2    F         396.63
              M         466.86
Special L3    F         495.57
              M         404.75
Name: Total Claim Amount, dtype: float64

Of the customers who have responded "Yes" to the last marketing campaign,\
the highest average total claim amount is 495.57, coming from female clients with Special L3 policy.\
The lowest avg claim of 391.61 comes from male clients with  Corporate L3 policy.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [41]:
# Each of the 5 states has more than 500 customers.
customers = df.groupby("State")["Customer"].nunique()
customers_500 = customers[customers > 500]
customers_500

State
Arizona       1703
California    3150
Nevada         882
Oregon        2601
Washington     798
Name: Customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [23]:
df.groupby(["Education", "Gender"])["Customer Lifetime Value"].agg(["max", "min", "median"]).round(2)

max      min   median
Education            Gender                            
Bachelor             F       73225.96  1904.00  5640.51
                     M       67907.27  1898.01  5548.03
College              F       61850.19  1898.68  5623.61
                     M       61134.68  1918.12  6005.85
Doctor               F       44856.11  2395.57  5332.46
                     M       32677.34  2267.60  5577.67
High School or Below F       55277.45  2144.92  6039.55
                     M       83325.38  1940.98  6286.73
Master               F       51016.07  2417.78  5729.86
                     M       50568.26  2272.31  5579.10

Male clients with high school or below bring both the highest CLV = customer lifetime value (83325.38) and highest median CLV (6286.73).\
Male clients with a bachelor's degree bring both the lowest CLV (1904) and lowest median CLV (5548.03).\
There doesn't seem to be correlation between the CLV and education.\
With male clients, the CLV seems to be dropping as education level increases, but this is not steady.\
With female clients, the CLV follows the rise of education level until the master's degree, then goes in the opposite direction.


## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [156]:
df["Month"] = df["Effective To Date"].str.split("/").str[0]
pivot = df.pivot_table(index="State", columns="Month", values="Policy", aggfunc="count")
pivot

Month,1,2
State,,
Arizona,1008,929
California,1918,1634
Nevada,551,442
Oregon,1565,1344
Washington,463,425


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [150]:
top_3_states = df.groupby("State")["Policy"].count().sort_values(ascending=False).head(3).index
top_3_states = df[df["State"].isin(top_3_states)]
top_3_states.groupby(["State", "Month"]).size().unstack(fill_value=0)

Month,1,2
State,,
Arizona,1008,929
California,1918,1634
Oregon,1565,1344


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [169]:
pivot_table = df.pivot_table(index="Sales Channel", columns="Response", aggfunc="size")

melted = pivot_table.reset_index().melt(id_vars="Sales Channel", value_vars=["Yes", "No"], var_name="Response", value_name="Count")

total_responses = melted.groupby("Sales Channel")["Count"].sum().reset_index(name="Total")
response_rate = melted[melted["Response"] == "Yes"].merge(total_responses, on="Sales Channel")
response_rate["Response Rate"] = (response_rate["Count"] / response_rate["Total"] * 100).round(2)

response_rate[["Sales Channel", "Response Rate"]]

,Sales Channel,Response Rate
0,Agent,19.07
1,Branch,11.38
2,Call Center,10.98
3,Web,11.71
